In [13]:
import gensim
from gensim.models import KeyedVectors
from gensim.models.fasttext import FastText as FT_gensim
import pandas as pd
import numpy as np

In [14]:
# FT_gensim.load('../wordvectors/wiki-news-300d-1M-subword.vec.zip')
wiki_vec = KeyedVectors.load_word2vec_format('../wordvectors/wiki.en.vec', binary=False, limit=5000)
vocab = set(wiki_vec.vocab)

In [15]:
wiki_vec['hello'].size

300

In [382]:

class Wiki:
    def __init__(self,id,title, all_tokens=[]):
        self.id = id
        self.title = title
        self.all_tokens = all_tokens
        
    def init_revisions(self, revisions):
#         self.revisions = {revision["id"] : Revision(revision["id"],revision["timestamp"], revision["editor"]) for revision in revisions}
          self.revisions = pd.Series( {revision["id"] : 
                                       Revision(revision["id"],revision["timestamp"], revision["editor"]) for revision in revisions} )


    def add_all_token(self, all_tokens):
        self.all_tokens = all_tokens
        for token in self.all_tokens:
            self.revisions.loc[token["o_rev_id"]].added.add(token["token_id"])
            for in_revision in token["in"]:
                self.revisions.loc[in_revision].added.add(token["token_id"])
            for out_revision in token["out"]:
                self.revisions.loc[out_revision].removed.add(token["token_id"])
class Revision:
    def __init__(self, id, timestamp,editor):
        self.id = id
        self.timestamp = timestamp
        self.editor = editor
        self.added = set()
        self.removed = set()
    
        
        
    def deleted(self, to_rev):
        self.content["removed"] = self.content["token_id"].isin(to_rev.removed)
        end_pos = np.argwhere(np.ediff1d(np.pad(self.content["removed"].astype(np.int), (1,1), mode="constant", constant_values=0)) == -1) -1 
        start_pos = np.argwhere(np.ediff1d(np.pad(self.content["removed"].astype(np.int), (1,1), mode="constant", constant_values=0)) == 1)
        start_neighbour = start_pos - 1
        end_neighbour = end_pos + 1
        self.first_last_token_del = self.content["removed"].values[[0,-1]]
        if self.first_last_token_del[0]:
            start_pos[0] = end_pos[0]
            start_neighbour[0] = end_neighbour[0]
        if self.first_last_token_del[1]:
            end_pos[-1] = start_pos[-1]
            start_neighbour[-1] = end_neighbour[-1]
        self.deleted_object = pd.DataFrame(np.c_[ start_pos, end_pos, start_neighbour, end_neighbour ],
                                       columns=[ "del_start_pos", "del_end_pos", "left_neigh", "right_neigh",])
    
    def inserted_continuous_pos(self):
        self.content["added"] = self.content["token_id"].isin(self.added)
        end_pos = np.argwhere(np.ediff1d(np.pad(self.content["added"].astype(np.int), (1,1), mode="constant", constant_values=0)) == -1) -1 
        start_pos = np.argwhere(np.ediff1d(np.pad(self.content["added"].astype(np.int), (1,1), mode="constant", constant_values=0)) == 1)
        self.added_pos = np.c_[start_pos, end_pos]

    def inserted_neighbours(self):
        start_token_pos = self.added_pos[:,0] - 1
        end_token_pos = self.added_pos[:,1] + 1
        self.first_last_token = self.content["added"].values[[0,-1]]
        if self.first_last_token[0]:
            start_token_pos[0] = end_token_pos[0]
        if self.first_last_token[1]:
            end_token_pos[-1] = start_token_pos[-1]
        self.start_token_id = self.content["token_id"].values[start_token_pos]
        self.end_token_id = self.content["token_id"].values[end_token_pos]
    
    def create_change_object(self, to_rev):
        ins_left = np.argwhere(from_rev.content.token_id.isin(to_rev.start_token_id))
        ins_right = np.argwhere(from_rev.content.token_id.isin(to_rev.end_token_id))
        self.inserted_object = pd.DataFrame(np.c_[to_rev.added_pos, ins_left, ins_right],
                                       columns=["ins_start_pos", "ins_end_pos", "left_neigh", "right_neigh", ])

        self.change = pd.merge(self.inserted_object, self.deleted_object,how="outer", on=["left_neigh", "right_neigh"])
        self.change.fillna(0, inplace=True)
    
    
    def append_neighbour_vec(self,wiki_vec, epsilon_size):
        neighbour_df = self.change.apply(find_tokens, axis=1, args=(self, wiki_vec, 6))
        neighbour_df.columns= ["ins_tokens", "del_tokens", "left_neigh", "right_neigh", "left_token", "right_token"]
        self.change = pd.concat([self.change, neighbour_df], sort=False)
class Change:
    def __init__(self, token, start, end, left_context, right_context):
        self.token = token
        self.start = start
        self.end = end
        self.left = left_context
        self.right = right_context

In [383]:
#TO-Do:Refractor it
def find_tokens(change, revision, wiki_vec, epsilon_size):
        vocabs_pos = np.argwhere( revision.content["invocab"].values)
        left_neigh = vocabs_pos[vocabs_pos <= change["left_neigh"]][-epsilon_size:]
        right_neigh = vocabs_pos[vocabs_pos >= change["right_neigh"]][:epsilon_size]
        ins_slice = slice(int(change["ins_start_pos"]), int(change["ins_end_pos"]+1) )
        del_slice = slice(int(change["del_start_pos"]), int(change["del_end_pos"]+1) )
        left_token = from_rev.content.str.values[left_neigh]
        right_token = from_rev.content.str.values[right_neigh]
        ins_tokens = from_rev.content.str.values[ins_slice]
        del_tokens = from_rev.content.str.values[del_slice]
        return pd.Series([tuple(ins_tokens), tuple(del_tokens), tuple(left_neigh), tuple(right_neigh), tuple(left_token), tuple(right_token)])

In [384]:
%%time
test_wiki = Wiki(2345, "a test wiki", all_tokens=4)
test_wiki.init_revisions(revisons_list)
test_wiki.add_all_token(all_tokens_mama) 
epsilon_size = 6

CPU times: user 7.02 s, sys: 0 ns, total: 7.02 s
Wall time: 7.02 s


In [385]:
%%time
test_wiki.revisions.iloc[0].content = pd.DataFrame(rev_contents[0])
for i in range(1, len(rev_contents)):
    from_rev = test_wiki.revisions.iloc[i-1]
    to_rev = test_wiki.revisions.iloc[i]
    from_rev.deleted(to_rev)
    from_rev.content["invocab"] = from_rev.content["str"].isin(set(wiki_vec.vocab))
    to_rev.content = pd.DataFrame(rev_contents[i])
    to_rev.inserted_continuous_pos()
    to_rev.inserted_neighbours()
    from_rev.create_change_object(to_rev)
    from_rev.append_neighbour_vec( wiki_vec, epsilon_size)

CPU times: user 304 ms, sys: 0 ns, total: 304 ms
Wall time: 303 ms


In [386]:
change_objects = []
test_wiki.revisions[:14].apply(lambda revision: change_objects.append(revision.change))
change_df = pd.concat(change_objects, sort=False, keys=test_wiki.revisions.index[:14])

In [387]:
from_rev = test_wiki.revisions[revisons.id.tolist()[0]] 
to_rev = test_wiki.revisions[revisons.id.tolist()[4]]


In [326]:
def get_word_vecs(tokens):
    in_vocab_tokens = set(tokens) & set(wiki_vec.vocab)
    if in_vocab_tokens:
        return wiki_vec[in_vocab_tokens].sum(axis=0, keepdims=True)
    else:
        return np.zeros((1, wiki_vec.vector_size))

In [327]:
%%time
change_vecs_list = []
change_token_s = change_df["ins_tokens"] + change_df["del_tokens"]
change_token_s.apply(lambda token_set: ins_vecs_list.append(get_word_vecs(token_set)))

change_matrix = np.concatenate(ins_vecs_list, axis=0)


CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 24.4 ms


In [328]:
%%time
neigh_vecs_list = []
neighbour_s = change_df['left_token'] + change_df['right_token']
neighbour_s.apply(lambda token_set: neigh_vecs_list.append(get_word_vecs(token_set)))

neighbour_matrix = np.concatenate(neigh_vecs_list, axis=0)

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 23.5 ms


In [171]:
# def neighbours(change, revision, wiki_vec, epsilon_size):
#     vocabs_pos = np.argwhere( revision.content["invocab"].values)
#     left = vocabs_pos[vocabs_pos < change["ins_starting_pos"]][:epsilon_size]
#     right = vocabs_pos[vocabs_pos > change["ins_end_pos"]][:epsilon_size]
#     neighbor_tokens = from_rev.content.str.values[np.r_[left,right]]
#     neighbour_vec = wiki_vec[neighbor_tokens].sum(axis=0)
#     combined_neighbour_vec = np.r_[left, right, neighbor_tokens, neighbour_vec]
#     return combined_neighbour_vec  

In [547]:
test_wiki.revisions[3317921].content["str"].values[np.s_[start_pos,end_pos[1:]]]

NameError: name 'start_pos' is not defined

In [ ]:
test_wiki.revisions[3317921].content["str"].values[start_pos[0][0]:end_pos[1][0]+1]

In [168]:
np.compress(test_wiki.revisions[3317921].content["removed"].values, test_wiki.revisions[3317921].content["str"].values)

array(['an', '80', 'u', 's', '.', 'ian', 'television'], dtype=object)

In [170]:
np.take(test_wiki.revisions[3317921].content["str"].values, pos )

array([['s', '.'],
       ['.', 'show']], dtype=object)